In [1]:
import numpy as np
import pandas as pd

In [ ]:


# Check if files exist and download otherwise
files = ['/u1.base', '/u1.test', '/u.item', '/u.user']
fname = dataset
data_dir = 'data/' + fname

download_dataset(fname, files, data_dir)

In [12]:
sep = '\t'
dtypes = {
    'u_nodes': np.int32, 'v_nodes': np.int32,
    'ratings': np.float32, 'timestamp': np.float64}

In [13]:
filename_train = '../../dataset/ml-100k/u1.base'
filename_test = '../../dataset/ml-100k/u1.test'

data_train = pd.read_csv(
    filename_train, sep=sep, header=None,
    names=['u_nodes', 'v_nodes', 'ratings', 'timestamp'], dtype=dtypes)

data_test = pd.read_csv(
    filename_test, sep=sep, header=None,
    names=['u_nodes', 'v_nodes', 'ratings', 'timestamp'], dtype=dtypes)

data_array_train = data_train.as_matrix().tolist()
data_array_train = np.array(data_array_train)
data_array_test = data_test.as_matrix().tolist()
data_array_test = np.array(data_array_test)

/home/mcui/.conda/envs/mingbo-env/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/home/mcui/.conda/envs/mingbo-env/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [15]:
data_array_train.shape

(80000, 4)

In [24]:
data_array = np.concatenate([data_array_train, data_array_test], axis=0)

u_nodes_ratings = data_array[:, 0].astype(dtypes['u_nodes'])
v_nodes_ratings = data_array[:, 1].astype(dtypes['v_nodes'])
ratings = data_array[:, 2].astype(dtypes['ratings'])

In [18]:
def map_data(data):
    """
    Map data to proper indices in case they are not in a continues [0, N) range

    Parameters
    ----------
    data : np.int32 arrays

    Returns
    -------
    mapped_data : np.int32 arrays
    n : length of mapped_data

    """
    uniq = list(set(data))

    id_dict = {old: new for new, old in enumerate(sorted(uniq))}
    data = np.array(map(lambda x: id_dict[x], data))
    n = len(uniq)

    return data, id_dict, n

In [23]:
data_array

array([[1.00000000e+00, 1.00000000e+00, 5.00000000e+00, 8.74965758e+08],
       [1.00000000e+00, 2.00000000e+00, 3.00000000e+00, 8.76893171e+08],
       [1.00000000e+00, 3.00000000e+00, 4.00000000e+00, 8.78542960e+08],
       ...,
       [4.59000000e+02, 9.34000000e+02, 3.00000000e+00, 8.79563639e+08],
       [4.60000000e+02, 1.00000000e+01, 3.00000000e+00, 8.82912371e+08],
       [4.62000000e+02, 6.82000000e+02, 5.00000000e+00, 8.86365231e+08]])

In [27]:
u_nodes_ratings, u_dict, num_users = map_data(u_nodes_ratings)
v_nodes_ratings, v_dict, num_items = map_data(v_nodes_ratings)

In [28]:
u_nodes_ratings, v_nodes_ratings = u_nodes_ratings.astype(np.int64), v_nodes_ratings.astype(np.int32)
ratings = ratings.astype(np.float64)

u_nodes = u_nodes_ratings
v_nodes = v_nodes_ratings

neutral_rating = -1  # int(np.ceil(np.float(num_classes)/2.)) - 1

# assumes that ratings_train contains at least one example of every rating type
rating_dict = {r: i for i, r in enumerate(np.sort(np.unique(ratings)).tolist())}

labels = np.full((num_users, num_items), neutral_rating, dtype=np.int32)
labels[u_nodes, v_nodes] = np.array([rating_dict[r] for r in ratings])

for i in range(len(u_nodes)):
    assert(labels[u_nodes[i], v_nodes[i]] == rating_dict[ratings[i]])

labels = labels.reshape([-1])

# number of test and validation edges, see cf-nade code

num_train = data_array_train.shape[0]
num_test = data_array_test.shape[0]
num_val = int(np.ceil(num_train * 0.2))
num_train = num_train - num_val

pairs_nonzero = np.array([[u, v] for u, v in zip(u_nodes, v_nodes)])
idx_nonzero = np.array([u * num_items + v for u, v in pairs_nonzero])

for i in range(len(ratings)):
    assert(labels[idx_nonzero[i]] == rating_dict[ratings[i]])

idx_nonzero_train = idx_nonzero[0:num_train+num_val]
idx_nonzero_test = idx_nonzero[num_train+num_val:]

pairs_nonzero_train = pairs_nonzero[0:num_train+num_val]
pairs_nonzero_test = pairs_nonzero[num_train+num_val:]

# Internally shuffle training set (before splitting off validation set)
rand_idx = range(len(idx_nonzero_train))
np.random.seed(42)
np.random.shuffle(rand_idx)
idx_nonzero_train = idx_nonzero_train[rand_idx]
pairs_nonzero_train = pairs_nonzero_train[rand_idx]

idx_nonzero = np.concatenate([idx_nonzero_train, idx_nonzero_test], axis=0)
pairs_nonzero = np.concatenate([pairs_nonzero_train, pairs_nonzero_test], axis=0)

val_idx = idx_nonzero[0:num_val]
train_idx = idx_nonzero[num_val:num_train + num_val]
test_idx = idx_nonzero[num_train + num_val:]

assert(len(test_idx) == num_test)

val_pairs_idx = pairs_nonzero[0:num_val]
train_pairs_idx = pairs_nonzero[num_val:num_train + num_val]
test_pairs_idx = pairs_nonzero[num_train + num_val:]

u_test_idx, v_test_idx = test_pairs_idx.transpose()
u_val_idx, v_val_idx = val_pairs_idx.transpose()
u_train_idx, v_train_idx = train_pairs_idx.transpose()

# create labels
train_labels = labels[train_idx]
val_labels = labels[val_idx]
test_labels = labels[test_idx]

if testing:
    u_train_idx = np.hstack([u_train_idx, u_val_idx])
    v_train_idx = np.hstack([v_train_idx, v_val_idx])
    train_labels = np.hstack([train_labels, val_labels])
    # for adjacency matrix construction
    train_idx = np.hstack([train_idx, val_idx])

# make training adjacency matrix
rating_mx_train = np.zeros(num_users * num_items, dtype=np.float32)
rating_mx_train[train_idx] = labels[train_idx].astype(np.float32) + 1.
rating_mx_train = sp.csr_matrix(rating_mx_train.reshape(num_users, num_items))

class_values = np.sort(np.unique(ratings))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'map'

In [2]:
sep = r'|'
movie_file = '../../dataset/ml-100k/u.item'
movie_headers = ['movie id', 'movie title', 'release date', 'video release date',
                 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
                 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                 'Thriller', 'War', 'Western']
movie_df = pd.read_csv(movie_file, sep=sep, header=None,
                       names=movie_headers, engine='python')

In [4]:
genre_headers = movie_df.columns.values[6:]
num_genres = genre_headers.shape[0]

In [29]:
v_features = np.zeros((1682, num_genres), dtype=np.float32)
for movie_id, g_vec in zip(movie_df['movie id'].values.tolist(), movie_df[genre_headers].values.tolist()):
    # check if movie_id was listed in ratings file and therefore in mapping dictionary
    if movie_id in v_dict.keys():
        v_features[v_dict[movie_id], :] = g_vec

In [33]:
v_features[0]

array([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [37]:
# user features

sep = r'|'
users_file = '../../dataset/ml-100k/u.user'
users_headers = ['user id', 'age', 'gender', 'occupation', 'zip code']
users_df = pd.read_csv(users_file, sep=sep, header=None,
                       names=users_headers, engine='python')

In [45]:
users_df

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [42]:
occupation = set(users_df['occupation'].values.tolist())

age = users_df['age'].values
age_max = age.max()

In [43]:
gender_dict = {'M': 0., 'F': 1.}
occupation_dict = {f: i for i, f in enumerate(occupation, start=2)}

In [47]:
len(occupation_dict)

21

In [44]:
num_feats = 2 + len(occupation_dict)

u_features = np.zeros((num_users, num_feats), dtype=np.float32)

In [46]:
u_features.shape

(943, 23)

In [34]:
for _, row in users_df.iterrows():
    u_id = row['user id']
    if u_id in u_dict.keys():
        # age
        u_features[u_dict[u_id], 0] = row['age'] / np.float(age_max)
        # gender
        u_features[u_dict[u_id], 1] = gender_dict[row['gender']]
        # occupation
        u_features[u_dict[u_id], occupation_dict[row['occupation']]] = 1.

KeyError: 0